In [4]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

In [5]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [10]:
n_eig = 6
nmax_r = 10
nmax_f = 15

only_inner = True
compensate_extra_cap = False

In [11]:
CR_1, CF_1, LF_1, LR_1, EJ_1, Δ_1, ω_r_1 = sq_ext.get_experimental_parameters('qubit_1')
CR_2, CF_2, LF_2, LR_2, EJ_2, Δ_2, ω_r_2 = sq_ext.get_experimental_parameters('qubit_2')
CR_3, CF_3, LF_3, LR_3, EJ_3, Δ_3, ω_r_3 = sq_ext.get_experimental_parameters('qubit_3')

In [ ]:
C_mat = sq_ext.C_mat_qubit_C_qubit(CC, CR_1, CF_1, CR_2, CF_2, only_inner=only_inner, compensate_extra_cap=compensate_extra_cap)
C_inv = np.linalg.inv(C_mat)
CF_1_tilde, CR_1_tilde, CF_2_tilde, CR_2_tilde = np.diag(C_inv) ** -1

F1 = sq_ext.sq_fluxonium(C_F_eff=CF_1_tilde, L_F_eff=LF_1, Δ=Δ_1, EJ=EJ_1, nmax_f=nmax_f)
R1 = sq_ext.sq_resonator(C_R_eff=CR_1_tilde, L_R_eff=LR_1, Δ=Δ_1, EJ=EJ_1, nmax_r=nmax_r)
F2 = sq_ext.sq_fluxonium(C_F_eff=CF_2_tilde, L_F_eff=LF_2, Δ=Δ_2, EJ=EJ_2, nmax_f=nmax_f)
R2 = sq_ext.sq_resonator(C_R_eff=CR_2_tilde, L_R_eff=LR_2, Δ=Δ_2, EJ=EJ_2, nmax_r=nmax_r)

H_F1 = F1.hamiltonian()
H_R1 = R1.hamiltonian()
    
# Move the coupler to the resonance point
φ_ext_resonance = sq_ext.find_resonance(H_F1, F2)
loop = F2.loops[0]
loop.set_flux(φ_ext_resonance)

H_F2 = F2.hamiltonian()
H_R2 = R2.hamiltonian()

I_F  = qt.identity(H_F1.dims[0])
I_R  = qt.identity(H_R1.dims[0])

Q_F1 = F1.charge_op(0)
Q_R1 = R1.charge_op(0)
Q_F2 = F2.charge_op(0)
Q_R2 = R2.charge_op(0)

H0_FF = qt.tensor(H_F1,I_F) + qt.tensor(I_F,H_F2)   
H0_FR = qt.tensor(H_F1,I_R) + qt.tensor(I_F,H_R2)
H0_RR = qt.tensor(H_R1,I_R) + qt.tensor(I_R,H_R2)

H_FF = H0_FF + C_inv[0,2] * fF**-1 * qt.tensor(Q_F1,Q_F2)   
H_FR = H0_FR + C_inv[0,3] * fF**-1 * qt.tensor(Q_F1,Q_R2)   
H_RR = H0_RR + C_inv[1,3] * fF**-1 * qt.tensor(Q_R1,Q_R2)

# Creating the basis as tensor product of individual hamiltonians

In [ ]:
ψ_0         = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,0])
ψ_1         = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,1])
ψ_0_prime   = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,0])
ψ_1_prime   = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,1])

ψ_0 = [qt.tensor(ψ_0, ψ_0_prime), qt.tensor(ψ_1,ψ_0_prime), qt.tensor(ψ_0,ψ_1_prime), qt.tensor(ψ_1,ψ_1_prime)]

H_eff_p1  = sq_ext.H_eff_p1 (H_0=None, H=H, n_eig=len(ψ_0), real=True, remove_ground=True, ψ_0=ψ_0)
H_eff_p2  = sq_ext.H_eff_p2 (H_0=H_0 , H=H, n_eig=len(ψ_0), real=True, remove_ground=True, ψ_0=ψ_0)
H_eff_SWT = sq_ext.H_eff_SWT(H_0=None, H=H, n_eig=len(ψ_0), real=True, remove_ground=True, ψ_0=ψ_0)

p1 [i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_p1,  print_pretty=False)
p2 [i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_p2,  print_pretty=False)
SWT[i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT, print_pretty=False)